# Visualizing Hu's Moments

In [18]:
import requests
import pandas as pd
import numpy as np
from skimage import io
from sklearn.decomposition import PCA
from scipy.spatial import ConvexHull
import bokeh
from bokeh.plotting import figure, show, output_notebook, output_file
import re

from IPython.display import Image, HTML, Javascript
from coffeetools import coffee
output_notebook()

Loading BokehJS ...

In [72]:
the_csv_file = 'file:///Users/Imperssonator/CC/DeepAFM/Hu_Imgs'
df = pd.read_csv(the_csv_file)
df.columns=(['path','H1','H2','H3','H4','H5','H6','H7'])
df['localName'] = ['file:///Users/Imperssonator/CC/DeepAFM/'+x for x in df['path']]
df['User']=[re.compile("Data\/capture\/(.*?)\/").search(x).group(1) for x in df['path']]
df.dropna(inplace=True)
df

,path,H1,H2,H3,H4,H5,H6,H7,localName,User
0,Data/capture/Anna/PEDOT_LiBTI.001_2.jpg,1.27100,2.373400e-02,6.974600e-01,0.807690,-4.106300e-02,-1.818700e-02,-1.942000e-02,file:///Users/Imperssonator/CC/DeepAFM/Data/ca...,Anna
1,Data/capture/Anna/PEDOT_LiBTI.001_3.jpg,0.99441,4.426700e-03,1.455000e-02,0.390380,1.494400e-03,2.688400e-03,8.198800e-04,file:///Users/Imperssonator/CC/DeepAFM/Data/ca...,Anna
2,Data/capture/Anna/PEDOT_LiBTI.002_1.jpg,7.45520,5.432220e+01,5.913800e+00,5.721500,8.982570e+01,4.009740e+01,2.363400e-01,file:///Users/Imperssonator/CC/DeepAFM/Data/ca...,Anna
3,Data/capture/Anna/PEDOT_LiBTI.002_2.jpg,11.92790,1.413534e+02,3.097390e+01,28.443000,2.330584e+03,3.270645e+02,1.255440e+01,file:///Users/Imperssonator/CC/DeepAFM/Data/ca...,Anna
4,Data/capture/Anna/PEDOT_LiBTI.002_3.jpg,24.62410,6.053582e+02,6.424530e+01,64.164900,1.227656e+04,1.573052e+03,7.991100e-01,file:///Users/Imperssonator/CC/DeepAFM/Data/ca...,Anna
5,Data/capture/Anna/PEDOT_LiBTI.003_1.jpg,1.55020,1.106700e-02,5.183700e-03,0.949980,3.175500e-04,-2.256400e-03,2.336300e-04,file:///Users/Imperssonator/CC/DeepAFM/Data/ca...,Anna
6,Data/capture/Anna/PEDOT_LiBTI.003_2.jpg,0.87737,6.438100e-04,6.067100e-03,0.202290,4.473200e-06,-6.429300e-06,3.119600e-07,file:///Users/Imperssonator/CC/DeepAFM/Data/ca...,Anna
7,Data/capture/Anna/PEDOT_LiBTI.003_3.jpg,0.78138,4.897400e-04,2.359800e-03,0.136420,-2.465700e-06,-1.668100e-05,-3.341800e-06,file:///Users/Imperssonator/CC/DeepAFM/Data/ca...,Anna
8,Data/capture/Anna/PEDOT_LiBTI.004_1.jpg,1.60940,1.948900e-02,1.324500e-03,1.064200,1.080700e-03,-4.137900e-03,1.767000e-04,file:///Users/Imperssonator/CC/DeepAFM/Data/ca...,Anna
9,Data/capture/Anna/PEDOT_LiBTI.004_2.jpg,0.87191,6.375600e-04,4.585800e-03,0.199670,8.655000e-07,-6.506500e-06,1.097000e-06,file:///Users/Imperssonator/CC/DeepAFM/Data/ca...,Anna


In [73]:
colors = {}
column_to_color = 'User'
for c in df[column_to_color].unique():
    colors[c] = np.random.randint( 0,255+1, (1,3))
    df.loc[df[column_to_color]==c,'color'] = '#' + ''.join([ '%02x'%s for s in colors[c][0]] )
    
df['color'].head()

0    #4a85bf
1    #4a85bf
2    #4a85bf
3    #4a85bf
4    #4a85bf
Name: color, dtype: object

In [74]:
df['localName'][0]

'file:///Users/Imperssonator/CC/DeepAFM/Data/capture/Anna/PEDOT_LiBTI.001_2.jpg'

In [75]:
df['H1'][1]

0.99441000000000002

In [80]:
df.loc[1:10]

,path,H1,H2,H3,H4,H5,H6,H7,localName,User,color
1,Data/capture/Anna/PEDOT_LiBTI.001_3.jpg,0.99441,0.004427,0.014550,0.39038,1.494400e-03,0.002688,8.198800e-04,file:///Users/Imperssonator/CC/DeepAFM/Data/ca...,Anna,#4a85bf
2,Data/capture/Anna/PEDOT_LiBTI.002_1.jpg,7.45520,54.322200,5.913800,5.72150,8.982570e+01,40.097400,2.363400e-01,file:///Users/Imperssonator/CC/DeepAFM/Data/ca...,Anna,#4a85bf
3,Data/capture/Anna/PEDOT_LiBTI.002_2.jpg,11.92790,141.353400,30.973900,28.44300,2.330584e+03,327.064500,1.255440e+01,file:///Users/Imperssonator/CC/DeepAFM/Data/ca...,Anna,#4a85bf
4,Data/capture/Anna/PEDOT_LiBTI.002_3.jpg,24.62410,605.358200,64.245300,64.16490,1.227656e+04,1573.052400,7.991100e-01,file:///Users/Imperssonator/CC/DeepAFM/Data/ca...,Anna,#4a85bf
5,Data/capture/Anna/PEDOT_LiBTI.003_1.jpg,1.55020,0.011067,0.005184,0.94998,3.175500e-04,-0.002256,2.336300e-04,file:///Users/Imperssonator/CC/DeepAFM/Data/ca...,Anna,#4a85bf
6,Data/capture/Anna/PEDOT_LiBTI.003_2.jpg,0.87737,0.000644,0.006067,0.20229,4.473200e-06,-0.000006,3.119600e-07,file:///Users/Imperssonator/CC/DeepAFM/Data/ca...,Anna,#4a85bf
7,Data/capture/Anna/PEDOT_LiBTI.003_3.jpg,0.78138,0.000490,0.002360,0.13642,-2.465700e-06,-0.000017,-3.341800e-06,file:///Users/Imperssonator/CC/DeepAFM/Data/ca...,Anna,#4a85bf
8,Data/capture/Anna/PEDOT_LiBTI.004_1.jpg,1.60940,0.019489,0.001324,1.06420,1.080700e-03,-0.004138,1.767000e-04,file:///Users/Imperssonator/CC/DeepAFM/Data/ca...,Anna,#4a85bf
9,Data/capture/Anna/PEDOT_LiBTI.004_2.jpg,0.87191,0.000638,0.004586,0.19967,8.655000e-07,-0.000007,1.097000e-06,file:///Users/Imperssonator/CC/DeepAFM/Data/ca...,Anna,#4a85bf
10,Data/capture/Anna/PEDOT_LiBTI.004_3.jpg,0.83032,0.000045,0.001881,0.14855,-9.384100e-06,0.000015,-5.021300e-06,file:///Users/Imperssonator/CC/DeepAFM/Data/ca...,Anna,#4a85bf


In [134]:
def zoom_img_plot(df,x_col,y_col,x_label='X',y_label='Y',url_col='file_url',plot_width = 800,plot_height=600):
    p=figure(
        responsive=False,
        x_axis_label = x_label,
        y_axis_label = y_label,
        width = plot_width,
        height= plot_height,
        active_scroll = 'wheel_zoom')
    
    num_rows = len(df.index)
    sub_sample = np.random.randint(1,high=num_rows,size=100).tolist()
    cds = bokeh.models.ColumnDataSource(
        df.iloc[sub_sample]
    )
    
    x_range = df[x_col].max()-df[x_col].min()
    y_range = df[y_col].max()-df[y_col].min()

    circles = p.circle(
        source=cds,
        x=x_col,
        y=y_col,
        fill_color='color',
        size=20,
        fill_alpha=.6
    )

    p.add_tools(
        bokeh.models.HoverTool(
            tooltips=[
                ('User', '@User')
            ]
        )
    )

    images = p.image_url(
        source=cds,
        x=x_col,
        y=y_col,
        w=256,
        h=256,
        h_units='screen',
        w_units='screen',
        url=url_col,
        anchor='center',
        global_alpha=.001
    )
    
#         w=x_range*0.0002,
#         h=y_range*0.0002*plot_width/plot_height/3,

    p.y_range.callback = bokeh.models.CustomJS( args= {
        'x_axis': p.x_range,
        'y_axis': p.y_range,
        'images': images.glyph,
        'cds': cds,
        'points': p.renderers[6].glyph
    }, 
    code=coffee.compile("""
    ddata = Bokeh._.max(cds.get('data')[points.get('x').field]) - Bokeh._.min(cds.get('data')[points.get('x').field])
    dx = x_axis.get('end') - x_axis.get('start')
    if dx/ddata > .0001
        images.set 'global_alpha', .001 + (.7)*Math.exp( -1*Math.pow(dx/ddata-.0001,2)/.014)
    """,bare=True)
    )

    p.axis.major_label_text_font_size='18pt'
    
    return p
    

In [133]:
p1 = zoom_img_plot(df,'H1','H3',x_label='Hu 1',y_label='Hu 2',url_col='localName')
output_notebook()
show(p1)

Loading BokehJS ...

ERROR://anaconda/envs/py35/lib/python3.5/site-packages/bokeh/core/validation/check.py:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: H1, H2, color [renderer: GlyphRenderer(id='533a9508-ce50-4117-ae8c-300ff93fb3c9', ...)]
ERROR://anaconda/envs/py35/lib/python3.5/site-packages/bokeh/core/validation/check.py:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: H1, H2, localName [renderer: GlyphRenderer(id='d72af5b9-02a6-4fd7-abea-94cd5c29f334', ...)]
ERROR://anaconda/envs/py35/lib/python3.5/site-packages/bokeh/core/validation/check.py:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: DotSize [renderer: GlyphRenderer(id='462796d3-d791-4996-b212-23b03a643835', ...)]


In [90]:
p1.renderers[5].glyph

Circle(id='f0ef8f53-a38c-4d6c-873e-378a0cf1abac', ...)

In [130]:
num_rows = len(df.index)
sub_sample = np.random.randint(1,high=num_rows,size=100)
sub_sample.tolist()
df.iloc[sub_sample]

,path,H1,H2,H3,H4,H5,H6,H7,localName,User,color
235,Data/capture/Boyi/091913/DHL180_150_1_Annealin...,0.60251,0.000010,0.000029,0.092663,-3.307700e-09,-1.661300e-07,3.407200e-09,file:///Users/Imperssonator/CC/DeepAFM/Data/ca...,Boyi,#9f1e3d
2164,Data/capture/Dalsu/9-8-2014/Cornelia/CU_P3HT_2...,1.42830,0.005010,0.033990,0.288000,5.945400e-04,-1.743700e-04,-9.051600e-04,file:///Users/Imperssonator/CC/DeepAFM/Data/ca...,Dalsu,#3644f1
2326,Data/capture/Daniel/sample 04.003_1.jpg,1.06860,0.001990,0.001812,0.283600,5.373000e-08,-5.281800e-05,2.198900e-06,file:///Users/Imperssonator/CC/DeepAFM/Data/ca...,Daniel,#dae9c3
2830,Data/capture/Guoyan/2016.03.07/p-2.500.030.005...,0.73888,0.000187,0.000216,0.146370,7.899100e-08,-2.570400e-06,3.012900e-08,file:///Users/Imperssonator/CC/DeepAFM/Data/ca...,Guoyan,#887dc2
1936,Data/capture/Dalsu/6-25-2014/0%_P3HT_0min_10um...,0.68616,0.000087,0.000100,0.119550,-4.510400e-10,-1.286200e-07,-7.991900e-10,file:///Users/Imperssonator/CC/DeepAFM/Data/ca...,Dalsu,#3644f1
1038,Data/capture/Dalsu/12-25-2014/Mincheol/NoUV50%...,1.37160,0.037646,0.100840,0.388380,3.853100e-06,-5.458200e-04,8.759300e-05,file:///Users/Imperssonator/CC/DeepAFM/Data/ca...,Dalsu,#3644f1
2766,Data/capture/Guoyan/2016.03.02/p-2-4-80.004_2.jpg,0.61790,0.000037,0.000022,0.095522,-3.298500e-10,5.064000e-08,5.053100e-10,file:///Users/Imperssonator/CC/DeepAFM/Data/ca...,Guoyan,#887dc2
826,Data/capture/Dalsu/11-25-2013/NSC-14(New)/CF_1...,0.90594,0.000303,0.001124,0.197350,-3.193200e-08,1.105900e-06,-3.000900e-08,file:///Users/Imperssonator/CC/DeepAFM/Data/ca...,Dalsu,#3644f1
8,Data/capture/Anna/PEDOT_LiBTI.004_1.jpg,1.60940,0.019489,0.001324,1.064200,1.080700e-03,-4.137900e-03,1.767000e-04,file:///Users/Imperssonator/CC/DeepAFM/Data/ca...,Anna,#4a85bf
2373,Data/capture/Gang/08042015/microfluidic printi...,0.55371,0.000003,0.000021,0.080665,2.203500e-09,-1.244200e-07,1.564900e-09,file:///Users/Imperssonator/CC/DeepAFM/Data/ca...,Gang,#a253dd
